## Projekt związany z wizualizacją danych MRI wraz z płaszczyzną ISO

## Instrukcja obsługująca wyświetalnie okna z wizualizacją

In [153]:
%gui qt

In [154]:
from mayavi import mlab
import numpy as np

## Pobranie danych 

In [155]:
import os
if not os.path.exists('MRbrain.tar.gz'):
    try:
        from urllib import urlopen
    except ImportError:
        from urllib.request import urlopen
    print("Downloading data, Please Wait (7.8MB)")
    opener = urlopen(
                'http://graphics.stanford.edu/data/voldata/MRbrain.tar.gz')
    open('MRbrain.tar.gz', 'wb').write(opener.read())

## Rozpakowanie danych

In [156]:
import tarfile
tar_file = tarfile.open('MRbrain.tar.gz')
try:
    os.mkdir('mri_data')
except:
    pass
tar_file.extractall('mri_data')
tar_file.close()

## Wczytanie danych jako trójwymiarowa tablica numpy

In [157]:
data = np.array([np.fromfile(os.path.join('mri_data', 'MRbrain.%i' % i),
                                        dtype='>u2') for i in range(1, 110)])
data.shape = (109, 256, 256)
data = data.T

## Wyświetlanie danych

In [158]:
mlab.figure(bgcolor=(0, 0, 0), size=(400, 400))
src = mlab.pipeline.scalar_field(data)

## Nasze dane nie są równomiernie rozmieszczone w przestrzeni trójwymiarowej

In [159]:
src.spacing = [1, 1, 1.5]
src.update_image_data = True

## Ekstrakcja wewnętrznych struktur: komory oraz włókna pomiędzy półkulami. Definiujemy volume of interest (VOI), które ograniczają izopowierzchnie do tych wewnątrz mózgu. Robimy to porzez wykorzystanie filtru ExtractGrid

In [160]:
blur = mlab.pipeline.user_defined(src, filter='ImageGaussianSmooth')
voi = mlab.pipeline.extract_grid(blur)
voi.trait_set(x_min=125, x_max=193, y_min=92, y_max=125, z_min=34, z_max=75)

mlab.pipeline.iso_surface(voi, contours=[1610, 2480], colormap='Spectral')


In [161]:
mlab.pipeline.scalar_cut_plane??

## Dodajemy dwie płaszczyzny aby pokazać nie obrobione dane MRI. Używamy filtru aby obciąć powirzchnie poza mózgiem.

In [163]:
thr = mlab.pipeline.threshold(src, low=1120)
cut_plane = mlab.pipeline.scalar_cut_plane(thr,
                                plane_orientation='y_axes',
                                colormap='black-white',
                                vmin=1400,
                                vmax=2600)
cut_plane.implicit_plane.origin = (136, 111.5, 82)
cut_plane.implicit_plane.widget.enabled = False

cut_plane2 = mlab.pipeline.scalar_cut_plane(thr,
                                plane_orientation='z_axes',
                                colormap='black-white',
                                vmin=1400,
                                vmax=2600)
cut_plane2.implicit_plane.origin = (136, 111.5, 82)
cut_plane2.implicit_plane.widget.enabled = False

## Wydobywamy dwa widoki powierzchni zewnętrznej. Potrzebujemy zdefiniować VOI aby zostawić wycięcie w głowie.

In [ ]:
voi2 = mlab.pipeline.extract_grid(src)
voi2.trait_set(y_min=112)
outer = mlab.pipeline.iso_surface(voi2, contours=[1776, ],
                                        color=(0.8, 0.7, 0.6))

voi3 = mlab.pipeline.extract_grid(src)
voi3.trait_set(y_max=112, z_max=53)
outer3 = mlab.pipeline.iso_surface(voi3, contours=[1776, ],
                                         color=(0.8, 0.7, 0.6))


mlab.view(-125, 54, 326, (145.5, 138, 66.5))
mlab.roll(-175)

mlab.show()

import shutil
shutil.rmtree('mri_data')